#### Now I try to play with the pixel to pixel radial gradient of Balmer decrement


In [3]:
from    astropy.table       import Table
from    scripts.tools       import *
import  numpy               as     np
from    astropy.io          import fits
from    astropy.cosmology   import Planck18
import  astropy.units       as     u
import  matplotlib.pyplot   as     plt
import  matplotlib.colors   as     colors  
import matplotlib
#matplotlib.use('Agg')
from    functools           import reduce
from    tqdm                import tqdm
import  os
import  gc                                         
import warnings
warnings.filterwarnings("ignore")
# Avoid division by zero
np.errstate(divide='ignore', invalid='ignore')
warnings.filterwarnings("ignore", message="marker is redundantly defined")



In [ ]:

#calculate distance of each pixel to center:
def image_to_center_distance(pixel_length):
    y,x = np.indices((50,50))
    center = (np.array((50,50)) - 1) / 2
    distance = np.sqrt((x - center[1])**2 + (y - center[0])**2)
    return (distance*pixel_length).value

#this is the joint function that:
#1. calculate the 2d map of balmer decrement + error
#calculate each pixel distance using redshift + return a table of balmer decrement vs distance
def extract_radial_profile(obj,
                            LINE_HA='LINE_HA',LINE_HB='LINE_HB_CONV',
                            LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB_CONV'):
    
    #load corresponding extracted file
    path = f"data_extracted/{file_name(obj,'extracted')}"
    with fits.open(path,mode='update') as extracted:

        #seg ha hb from extracted data
        seg = find_data('SEG',extracted)[1].data == obj['ID']
        ha  = find_data(LINE_HA,extracted)[1]
        hb  = find_data(LINE_HB,extracted)[1]
        err_ha = find_data(LINEWHT_HA,extracted)[1].data**(-0.5)
        err_hb = find_data(LINEWHT_HB,extracted)[1].data**(-0.5)
        #this mask out all the the negative pixel due to continium oversubtraction
        mask1 = np.logical_and(find_data('LINE_HA',extracted)[1].data>0, find_data('LINE_HB',extracted)[1].data>0)
        mask2 = np.logical_and(ha.data>0,hb.data>0)
        mask  = np.logical_and(np.logical_and(mask1,mask2),seg)

        #pixel length from redshift, !! here angular diameter distance is used
        #to  center distance
        #------------------------------------------------prob no need change to rad




        pixel_length = np.deg2rad(ha.header['PIXASEC']/3600) * Planck18.angular_diameter_distance(obj['z_MAP']).to(u.kpc)






        distances_obj  = image_to_center_distance(pixel_length)[mask]
        sort = np.argsort(distances_obj)
        #this gives the distance sorted out.
        distances_obj = distances_obj[sort]
        
        #this part calculate the actual 2d balmer decrement + error
        balmer_2d     = fits.ImageHDU(data = ha.data/hb.data,name = '2D_BALMER')
        
        #!!!! note the error here is just using the square root of the image thumbnail, weightmap needed for improvement!!
        balmer_2d_err = fits.ImageHDU(data = (err_ha**2/hb.data**2 + ha.data**2 * err_hb**2/hb.data**2)**0.5,name = '2D_BALMER_ERR')


        ha_pix = ha.data[mask][sort]
        hb_pix = hb.data[mask][sort]
        ha_pix_err = err_ha[mask][sort]
        hb_pix_err = err_hb[mask][sort]
        balmer_pix     = balmer_2d.data[mask][sort]
        balmer_pix_err = balmer_2d_err.data[mask][sort]

        dtype = [('DISTANCE [kpc]', 'f4'), ('ha', 'f4'), ('ha_err', 'f4'), ('hb', 'f4'), ('hb_err', 'f4'), ('balmer', 'f4'), ('balmer_err', 'f4')]
        name = f'PIX_RAD_PROFILE'
        balmer_table = fits.BinTableHDU(np.array(list(zip(distances_obj, ha_pix, ha_pix_err, hb_pix, hb_pix_err, balmer_pix, balmer_pix_err)), dtype=dtype), name=name)
        #this is the distance sorted out


        #save update
        #save_update(balmer_2d,extracted)
        #save_update(balmer_2d_err,extracted)
        #save_update(balmer_table,extracted)
        #extracted.flush()
        balmer_table.writeto(f"data_extracted/{file_name(obj,prefix='rad')}", overwrite=True)
    return f"{obj['subfield']}-{obj['ID']} processed"


from    concurrent.futures  import ThreadPoolExecutor, as_completed
def cat_process(obj_lis, max_threads=1):
        results = []
        if max_threads > 1 :
            with ThreadPoolExecutor(max_threads) as executor:
                futures = {executor.submit(
                    extract_radial_profile,
                    obj): obj for obj in obj_lis}
                for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                    results.append(future.result())
            return results
        else:
            for obj in tqdm(obj_lis):
                results.append(extract_radial_profile(obj))
            return results

def main():
    obj_lis = Table.read('obj_lis_selected.fits')
    keep = obj_lis['manual_select']=='keep'
    results = cat_process(obj_lis[keep], max_threads=1)
    errorcounting(results)

if __name__ == '__main__' :
    main()

In [ ]:
import os
import numpy as np
from astropy.io import fits
from astropy.table import Table
from astropy.wcs import WCS
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from matplotlib import colors
from matplotlib.pyplot import cm
from matplotlib import use
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb

%matplotlib inline
import matplotlib.pyplot as plt
from astropy.visualization import (MinMaxInterval, SqrtStretch, ImageNormalize)
from scripts.tools import *


def plot_balmer_decrem(obj, plot, crop_size=50,sn_threshold=1):
    try:
        # Construct the path to the FITS file
        path = f"data_extracted/{file_name(obj, prefix='extracted')}"
        
        # Open the FITS file
        with fits.open(path) as hdu:
            # Determine the center of the image
            shape = hdu[3].data.shape[0]
            si = (shape - crop_size) // 2; 
            ei = si + crop_size

            #extract segmentation map
            if find_data('SEG_MOD',hdu) != None:
                seg = find_data('SEG_MOD',hdu)[1].data
            else:
                seg = find_data('SEG',hdu)[1].data

            #effective radius (in arcsec)
            r_eff = obj['re']
             
            # Create a figure with subplots
            fig, axes = plt.subplots(2, 4, figsize=(14, 7))
            #set default tick label blank
            for ax in axes.flatten():
                ax.tick_params(labelbottom=False, labelleft=False)
                ax.set_xticklabels([])
                ax.set_yticklabels([])
            axes = axes.flatten()
            #set default fontsize
            plt.rcParams.update({'font.size': 14})

            # Loop through the specified image names and plot them
            #here i try to add segmentation map effect to the image
            for i, name in enumerate(['DSCI', 'LINE_HA', 'LINE_HB_CONV', '2D_BALMER']):
                image = find_data(name, hdu)[1]
                titles = {
                        "DSCI": 'direct imaging',
                        "LINE_HA": r"H$\alpha$",
                        "LINE_HB_CONV": r"H$\beta$",
                        "2D_BALMER": r'$H\alpha / H\beta$'
                        }
                data = image.data[si:ei, si:ei]
                # Extract the segmentation map for the same region
                seg_crop = seg[si:ei, si:ei]
                # Create a mask where the segmentation map matches the object ID
                mask = seg_crop == obj['ID']
                
                ax = axes[i]
                ax.tick_params(direction='in',which='both', top=True, right=True)

                # Plot the original data with plasma_r colormap
                norm = colors.LogNorm()#vmin=np.nanmin(data),vmax = np.nanmax(data))
                if name == '2D_BALMER':
                    norm = colors.Normalize(vmin=0, vmax=10)
                im = ax.imshow(np.where(mask,data,np.nan), norm=norm, origin='lower', cmap='plasma_r')
                
                # Convert the colormap to grayscale
                cmap = cm.get_cmap('plasma_r')
                cmap_colors = cmap(np.arange(cmap.N))
                hsv_colors = rgb_to_hsv(cmap_colors[:, :3])
                hsv_colors[:, 1] = 0  # Set saturation to 0 to convert to grayscale
                gray_cmap = colors.ListedColormap(hsv_to_rgb(hsv_colors))

                # Overlay the segmentation map with gray colormap and lower transparency
                ax.imshow(np.where(np.logical_not(mask),data,np.nan), norm=norm, origin='lower', cmap=gray_cmap, alpha=0.5)
                
                # Plot a circle representing the effective radius
                circ = plt.Circle((crop_size/2-1, crop_size/2-1), r_eff / 0.1, color='blue', fill=False, linestyle='--',label='effective radius',linewidth=2)
                ax.add_patch(circ)

                ax.plot([3, 7], [4, 4])
                ax.text(5, 5, f"{round(obj['pixel_length'] * 4, 2)} kpc")
                ax.set_ylabel(f'{titles[name]}')
                ax.set_xticklabels([]);ax.set_yticklabels([])
                ax.legend()

            # Extract and plot the radial profiles
            table = Table.read(f"data_extracted/{file_name(obj, prefix='rad')}")
            mask1 = table['ha']/table['ha_err'] > sn_threshold
            mask2 = table['hb']/table['hb_err'] > sn_threshold
            mask = np.logical_and(mask1,mask2)
            table = table[mask]
            r = table['DISTANCE [kpc]']
            ha_r = table['ha']
            ha_r_err = table['ha_err']
            hb_r = table['hb']
            hb_r_err = table['hb_err']
            balmer_r = table['balmer']
            balmer_r_err = table['balmer_err']
            A_Ha = -2.5 * np.log10(ha_r / hb_r)


            if True:
                # Plot the Ha and Hb radial profiles
                ax = plt.subplot2grid((2, 4), (1, 0), colspan=2)
                ax.errorbar(r, ha_r, yerr=ha_r_err, fmt='ro:', label=r'H$\alpha$', alpha=0.5,linewidth=0.001,elinewidth=1)
                ax.errorbar(r, hb_r, yerr=hb_r_err, fmt='go:', label=r'H$\beta$', alpha=0.5,linewidth=0.001,elinewidth=1)

                # Effective radius
                ax.axvspan(0, r_eff * obj['pixel_length'] / 0.1, color='grey', alpha=0.3)
                
                # Determine the plot limits
                arr = np.array((ha_r, hb_r))
                arr = arr[arr > 0]
                plot_min = np.nanmin(arr) * 0.05
                plot_max = np.nanmax(arr) * 100

                # Annotate the effective radius
                ax.annotate(
                    "",  # Only draw the arrow, no text
                    xy=(0, plot_min * 2),  # Right endpoint
                    xytext=(r_eff * obj['pixel_length'] / 0.1, plot_min * 2),  # Left endpoint
                    arrowprops=dict(arrowstyle="<->", lw=1.5, color="black"),  # Horizontal double arrow
                )
                ax.text(r_eff * obj['pixel_length'] / 0.1 / 2, plot_min * 2.1, 
                        r"$r_{eff}$", fontsize=12, color="black", ha="center",va='bottom')
                ax.set_xlabel('r [kpc]')
                ax.set_ylabel('flux [$10^{-17}$ erg/s/cm$^2$]')
                ax.set_ylim(plot_min, plot_max)
                ax.set_yscale('log')
                ax.tick_params(direction='in', which='both', top=True, right=True)
                ax.legend()

                # Adjust y-axis tick labels to avoid overlap
                ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0e}'.format(y)))

            if True:
                # Plot the Balmer decrement radial profile
                #make xticks no overlap
                ax = plt.subplot2grid((2, 4), (1, 2), colspan=2)
                mask = np.logical_and(np.logical_not(np.isnan(balmer_r)),balmer_r>0)
                ax.set_xlabel('distance [kpc]')
                ax.set_ylabel(r'$H\alpha / H\beta$')
                ax.set_ylim(-5, 15)
                ax.set_xlim(0,3*r_eff * obj['pixel_length'] / 0.1)
                # Annotate the effective radius
                plot_min = -4.5
                #effective radius
                ax.axvspan(0, r_eff * obj['pixel_length'] / 0.1, color='grey', alpha=0.3)
                ax.annotate(
                    "",  # Only draw the arrow, no text
                    xy=(0, plot_min),  # Right endpoint
                    xytext=(r_eff * obj['pixel_length'] / 0.1, plot_min),  # Left endpoint
                    arrowprops=dict(arrowstyle="<->", lw=1.5, color="black"),  # Horizontal double arrow
                )
                ax.text(r_eff * obj['pixel_length'] / 0.1 / 2, plot_min, 
                        r"$r_{eff}$", fontsize=12, color="black", ha="center",va='bottom')

                # Plot the A_Ha radial profile on a secondary y-axis
                ax2 = ax.twinx()
                l2 = ax2.errorbar(r[mask], A_Ha[mask], yerr=balmer_r_err[mask], color='darkblue', linestyle='-', fmt='o', label=r'Attenuation H$\alpha$', alpha=0.5,linewidth=0.001,elinewidth=1)
                ax2.set_ylabel(r'$A_{H\alpha}$')
                ax2.set_ylim(-5,5)
                ax.set_xlim(0,3*r_eff * obj['pixel_length'] / 0.1)
                # Combine legends
                lines, labels = ax.get_legend_handles_labels()
                lines2, labels2 = ax2.get_legend_handles_labels()
                ax2.legend(lines + lines2, labels + labels2, loc='upper left')

                ax.tick_params(direction='in',which='both', top=True, right=False)
                ax2.tick_params(direction='in',right=True)

            # Add a big title for the entire figure
            fig.suptitle(f"{obj['subfield']}- {obj['ID']}, Mass: {round(np.log10(obj['mass']),2)}[log$M_\odot$], Redshift: {round(obj['z_MAP'],2)}, $r_e: {round(r_eff/0.1)} [kpc]$", fontsize=16)

            # Save the plot
            save_path = f"radial_balmer_decrem/{plot}"
            save_path_sn_10 = f"sn_10/radial_balmer_decrem/{plot}"
            os.makedirs(save_path, exist_ok=True)
            os.makedirs(save_path_sn_10, exist_ok=True)
            plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to make room for the big title

            if obj['sn_hb'] > 10:
                plt.savefig(f"{save_path_sn_10}/{obj['subfield']}-{obj['ID']}_{obj['tag']}_{obj['manual_select']}.png",dpi=300)
            if obj['manual_select'] == 'keep':
                plt.savefig(f"{save_path}/{obj['subfield']}-{obj['ID']}_{obj['tag']}_{obj['manual_select']}.png")
            else:
                plt.savefig(f"{save_path}/selected/{obj['subfield']}-{obj['ID']}_{obj['tag']}_{obj['manual_select']}.png")

                
           #plt.close('all')

            return f"{obj['subfield']}-{obj['ID']} saved"
    except Exception as e:
            print(f"! {obj['subfield']}-{obj['ID']} failed, error{e}")
            return f"! {obj['subfield']}-{obj['ID']} failed, error{e}"


def cat_process(obj_lis, plot='RA D_PROFILE', max_threads=1):
    print(f'\n start plot process{plot}')
    results = []
    if max_threads > 1:
        # Use multithreading to process the objects in parallel
        with ThreadPoolExecutor(max_threads) as executor:
            futures = {executor.submit(plot_balmer_decrem, obj, plot=plot): obj for obj in obj_lis}
            for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                results.append(future.result())
        return results
    else:
        # Process the objects sequentially
        for obj in tqdm(obj_lis):
            results.append(plot_balmer_decrem(obj, plot))
        return results

def main():
    use('Agg')  # Use the 'Agg' backend for matplotlib
    obj_lis = Table.read('obj_lis_selected.fits')
    keep = obj_lis['manual_select']=='keep'
    # Process the objects and plot the results
    results1 = cat_process(obj_lis[keep], plot='RAD_PROFILE', max_threads=1)
    errorcounting(results1)
    print(results1)

if __name__ == '__main__':
    main()



 start plot processRAD_PROFILE


  0%|          | 0/934 [00:00<?, ?it/s]/var/folders/6h/pm731bh5769gqlt1mrys7wth0000gn/T/ipykernel_50355/2506853569.py:77: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('plasma_r')
/var/folders/6h/pm731bh5769gqlt1mrys7wth0000gn/T/ipykernel_50355/2506853569.py:77: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('plasma_r')
/var/folders/6h/pm731bh5769gqlt1mrys7wth0000gn/T/ipykernel_50355/2506853569.py:77: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.ge

! GN2-9706 failed, error'NoneType' object is not subscriptable
! GN2-9748 failed, error'NoneType' object is not subscriptable


/var/folders/6h/pm731bh5769gqlt1mrys7wth0000gn/T/ipykernel_50355/2506853569.py:77: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('plasma_r')
/var/folders/6h/pm731bh5769gqlt1mrys7wth0000gn/T/ipykernel_50355/2506853569.py:77: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('plasma_r')
/var/folders/6h/pm731bh5769gqlt1mrys7wth0000gn/T/ipykernel_50355/2506853569.py:77: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cma

! GN2-10172 failed, error'NoneType' object is not subscriptable
! GN7-10272 failed, error'NoneType' object is not subscriptable
! GN2-10284 failed, error'NoneType' object is not subscriptable
! GN2-10336 failed, error'NoneType' object is not subscriptable


...

/var/folders/6h/pm731bh5769gqlt1mrys7wth0000gn/T/ipykernel_50355/2506853569.py:77: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('plasma_r')
/var/folders/6h/pm731bh5769gqlt1mrys7wth0000gn/T/ipykernel_50355/2506853569.py:77: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('plasma_r')
 18%|█▊        | 170/934 [00:26<01:53,  6.71it/s]/var/folders/6h/pm731bh5769gqlt1mrys7wth0000gn/T/ipykernel_50355/2506853569.py:77: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or `

! GN4-20432 failed, error'NoneType' object is not subscriptable
! GN4-20464 failed, error'NoneType' object is not subscriptable
! GN4-20481 failed, error'NoneType' object is not subscriptable


/var/folders/6h/pm731bh5769gqlt1mrys7wth0000gn/T/ipykernel_50355/2506853569.py:77: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('plasma_r')
/var/folders/6h/pm731bh5769gqlt1mrys7wth0000gn/T/ipykernel_50355/2506853569.py:77: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('plasma_r')
/var/folders/6h/pm731bh5769gqlt1mrys7wth0000gn/T/ipykernel_50355/2506853569.py:77: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cma

! GN4-20531 failed, error'NoneType' object is not subscriptable
! GN7-20579 failed, error'NoneType' object is not subscriptable
! GN4-20602 failed, error'NoneType' object is not subscriptable


/var/folders/6h/pm731bh5769gqlt1mrys7wth0000gn/T/ipykernel_50355/2506853569.py:77: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('plasma_r')
/var/folders/6h/pm731bh5769gqlt1mrys7wth0000gn/T/ipykernel_50355/2506853569.py:77: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('plasma_r')
 19%|█▉        | 176/934 [00:27<01:17,  9.80it/s]/var/folders/6h/pm731bh5769gqlt1mrys7wth0000gn/T/ipykernel_50355/2506853569.py:77: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or `

In [5]:
#mconvert each row of obj_lis into an clss Object
class Object:
    def __init__(self, row):
        self.subfield = row['subfield']
        self.ID = row['ID']
        self.z_MAP = row['z_MAP']
        self.mass = row['mass']
        self.re = row['re']
        self.pixel_length = row['pixel_length']
        self.sn_hb = row['sn_hb']
        self.tag = row['tag']
        self.manual_select = row['manual_select']

    def __repr__(self):
        return f"Object({self.subfield}, {self.ID})"
#make a small test using obj_lis[0] to print different parameters
def test_object(obj_lis):
    obj = Object(obj_lis[0])
    print(f"Subfield: {obj.subfield}")
    print(f"ID: {obj.ID}")
    print(f"Redshift (z_MAP): {obj.z_MAP}")
    print(f"Mass: {obj.mass}")
    print(f"Effective Radius (re): {obj.re}")
    print(f"Pixel Length: {obj.pixel_length}")
    print(f"S/N Hβ: {obj.sn_hb}")
    print(f"Tag: {obj.tag}")
    print(f"Manual Select: {obj.manual_select}")
if __name__ == '__main__':
    obj_lis = Table.read('obj_lis_selected.fits')
    test_object(obj_lis)
    #test_object(obj_lis[0])

Subfield: GN2
ID: 10512
Redshift (z_MAP): 1.268630364478613
Mass: 70250168313.80606
Effective Radius (re): 0.279708
Pixel Length: 0.8578072705548977
S/N Hβ: 11.88317709904857
Tag: agn
Manual Select: keep
